In [ ]:
import pandas as pd

gen_file = '/content/drive/MyDrive/Portfolio/Energy/energy.csv'
weather_file = '/content/drive/MyDrive/Portfolio/Energy/weather_features.csv'

#load datasets
df_gen = pd.read_csv(gen_file)
df_weather = pd.read_csv(weather_file)


In [ ]:
print(df_weather.columns)

Index(['dt_iso', 'city_name', 'temp', 'temp_min', 'temp_max', 'pressure',
       'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h',
       'clouds_all', 'weather_id', 'weather_main', 'weather_description',
       'weather_icon'],
      dtype='object')


In [ ]:
print(df_gen.columns)

Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage aggregated',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
       'total load forecast', 'total load actual', 'price day ahead',
       'price actual'],
      dtype='object')


In [ ]:
#handle datetime

#rename columns
df_gen.rename(columns={'time': 'datetime'}, inplace=True)
df_weather.rename(columns={'dt_iso': 'datetime'}, inplace=True)

#convert to datetime with UTC
df_gen['datetime'] = pd.to_datetime(df_gen['datetime'], utc=True)
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'], utc=True)

#remove timezone for Tableau
df_gen['datetime'] = df_gen['datetime'].dt.tz_localize(None)
df_weather['datetime'] = df_weather['datetime'].dt.tz_localize(None)

In [ ]:
#merge datasets on datetime
df = pd.merge(df_gen, df_weather, on='datetime', how='left')

In [ ]:
#filling missing values

#generation columns
generation_cols = [col for col in df_gen.columns if col.startswith('generation')]
df[generation_cols] = df[generation_cols].fillna(0)

#weather columns
weather_cols = ['temp','temp_min','temp_max','pressure','humidity','wind_speed','wind_deg',
                'rain_1h','rain_3h','snow_3h','clouds_all']
df[weather_cols] = df[weather_cols].fillna(df[weather_cols].mean())

In [ ]:
#convert temp from Kelvin to Celsius
for col in ['temp','temp_min','temp_max']:
    df[col] = df[col] - 273.15

In [ ]:
#calculated fields

#total generation
df['total_generation'] = df[generation_cols].sum(axis=1)

#renewable share
renewable_cols = [
    'generation solar', 'generation wind onshore', 'generation wind offshore',
    'generation other renewable', 'generation hydro run-of-river and poundage',
    'generation hydro water reservoir'
]
df['renewable_generation'] = df[renewable_cols].sum(axis=1)
df['renewable_share'] = df['renewable_generation'] / df['total_generation']

#demand supply gap
df['demand_supply_gap'] = df['total load actual'] - df['total_generation']


In [ ]:
#extract year, month, day, hour
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour

In [ ]:
#daily aggregation
sum_cols = generation_cols + ['total load actual', 'total_generation', 'demand_supply_gap', 'renewable_generation']
avg_cols = weather_cols + ['renewable_share']

df_daily = df.groupby(['year','month','day']).agg(
    {**{col:'sum' for col in sum_cols},
     **{col:'mean' for col in avg_cols}}
).reset_index()


In [ ]:
#recreate date column for Tableau
df_daily['date'] = pd.to_datetime(df_daily[['year','month','day']])
df_daily['month_year'] = df_daily['date'].dt.strftime('%b-%Y')  # e.g., Jan-2026

#save cleaned CSV ---
df_daily.to_csv('/content/energy_cleaned_daily_full_sources.csv', index=False)

In [ ]:
#month number to month name
df_daily['month_name'] = df_daily['date'].dt.strftime('%B')

#define function to map month to season
def month_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

 #create a new season column from function
df_daily['season'] = df_daily['month'].apply(month_to_season)

In [128]:
from google.colab import files

#save
df_daily.to_csv('/content/energy.csv', index=False)

#download
files.download('/content/energy.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(df_daily[['total_generation','temp','wind_speed']].describe())


       total_generation         temp   wind_speed
count      1.462000e+03  1462.000000  1462.000000
mean       3.493364e+06    16.506252     2.467029
std        3.974565e+05     6.464381     0.952099
min        1.412550e+05    -0.658537     0.925620
25%        3.231515e+06    11.115432     1.866942
50%        3.467030e+06    16.029022     2.241667
75%        3.732922e+06    22.148784     2.792958
max        5.690424e+06    29.665868     9.608333
